# 09 Walk Forward
This notebook performs a walk-forward evaluation of the trading strategy.


In [ ]:
import sys, pathlib
import pandas as pd
import numpy as np
import joblib
from src import config as cfg
from src import evol_utils as eu
from tensorflow import keras

PROJECT_ROOT = pathlib.Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


In [ ]:
# Load prices and features
df_prices = pd.read_parquet(cfg.DATA / 'raw' / 'prices.parquet').sort_index()
data = joblib.load(cfg.DATA / 'processed' / 'cnn5d_data.pkl')
tickers = data['tickers']
df_prices = df_prices[tickers]
df_ret = np.log(df_prices / df_prices.shift(1)).dropna()
ret5 = df_ret.rolling(5).sum()
vol5 = df_ret.rolling(5).std()
momentum = (ret5 / vol5).shift(1)
df_feat = pd.concat([df_ret.shift(1), momentum], axis=1).dropna()


In [ ]:
def _lazy_loader(var_name, pkl_path):
    if var_name not in globals():
        globals()[var_name] = joblib.load(pkl_path)
    return globals()[var_name]

def rebalancear_en_fecha(fecha, df_feat, model, w_prev=None):
    try:
        idx = df_feat.index.get_loc(fecha)
        ventana = df_feat.iloc[idx - cfg.WINDOW: idx]

        scaler_X = _lazy_loader('scaler_X_cnn5d', cfg.MODELS / 'scaler_X_cnn5d.pkl')
        X_scaled = scaler_X.transform(ventana.values)
        n_assets = len(tickers)
        X_input = X_scaled.reshape(1, cfg.WINDOW, n_assets, 2)
        r_hat = model.predict(X_input, verbose=0)[0]
        scaler_y = _lazy_loader('scaler_y_cnn5d', cfg.MODELS / 'scaler_y_cnn5d.pkl')
        r_hat = scaler_y.inverse_transform([r_hat])[0] / 5.0
        r_hat = np.clip(r_hat, -0.12, 0.12)

        fecha_ret = df_feat.index[idx]
        ventana_ret = df_ret.loc[:fecha_ret - pd.Timedelta(days=1)].tail(cfg.WINDOW)
        Sigma = ventana_ret.cov().values
        res = eu.resolver_optimizacion(r_hat, Sigma, w_prev=w_prev)
        w_star = eu.elegir_w_star(res, r_hat, Sigma, w_prev=w_prev)

        turnover = np.sum(np.abs(w_star - w_prev)) if w_prev is not None else 1.0
        ret_bruto = df_ret.iloc[idx: idx + cfg.REBAL_FREQ].values @ w_star
        ret_neto = ret_bruto.sum() - turnover * cfg.COST_TRADE
        ret_diarios = pd.Series(ret_bruto, index=df_ret.iloc[idx: idx + cfg.REBAL_FREQ].index)

        return {
            'fecha': fecha,
            'ret_neto': ret_neto,
            'w_star': w_star,
            'ret_diarios': ret_diarios
        }
    except Exception as e:
        print(f'ERROR {fecha.date()}: {e}')
        return None


In [ ]:
# Date ranges
periods = [
    ('2017-01-01', '2019-01-01'),
    ('2019-01-01', '2021-01-01'),
    ('2021-01-01', '2024-01-01')
]

all_returns = []
w_prev = None
model = keras.models.load_model(cfg.MODELS / 'cnn5d.keras', compile=False)

for start, end in periods:
    fechas = df_feat.loc[start:end].index
    for i in range(cfg.WINDOW, len(fechas) - cfg.REBAL_FREQ, cfg.REBAL_FREQ):
        fecha = fechas[i]
        out = rebalancear_en_fecha(fecha, df_feat, model, w_prev=w_prev)
        if out is not None:
            all_returns.append(out['ret_diarios'])
            w_prev = out['w_star']


In [ ]:
equity = (1 + pd.concat(all_returns)).cumprod()
joblib.dump(equity, cfg.RESULT / "walk_forward_equity.pkl")
equity.to_frame("equity").plot(figsize=(12,4))
